In [ ]:
from dataset.grammar import get_cfg, sample_programs, parse_program
from nltk.parse import RecursiveDescentParser
grammar = get_cfg()


parser = RecursiveDescentParser(grammar)

code = """
def program(a, b):
    if a < b:
        return a
    else:
        return b
"""
for tree in parser.parse(code.split()):
    print(tree)

ValueError: Grammar does not cover some of the input words: '\'program(a,\', \'b):\', \'b:\', \'else:\', \'b"\''.

(ALL
  (LEVEL0_1
    (INITIALIZATION (VARIABLE x) (EQUALS =) (DIGIT 10) (NEW_LINE \n))))


KeyboardInterrupt: 

In [ ]:
from dataset.grammar import sample_programs

sample_programs(n=10, level="LEVEL1_1")

['a=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=a+a\\nprint(a*g)',
 'a=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=a+a\\nprint(t)',
 'a=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=a+a\\nprint(v)',
 'a=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=a+a\\nprint(e)',
 'a=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=a+a\\nprint(a-u)',
 'a=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na

In [ ]:
from dataset.grammar import sample_programs

sample_programs(n=10, level="LEVEL1_1")

['a=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=a+a\\nprint(a*g)',
 'a=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=a+a\\nprint(t)',
 'a=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=a+a\\nprint(v)',
 'a=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=a+a\\nprint(e)',
 'a=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=a+a\\nprint(a-u)',
 'a=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na=0\\na

In [ ]:
from dataset.grammar import get_cfg
from pprint import pprint
grammar = get_cfg()

pprint(grammar.productions())

[VARIABLE -> 'a',
 VARIABLE -> 'b',
 VARIABLE -> 'c',
 VARIABLE -> 'd',
 VARIABLE -> 'e',
 VARIABLE -> 'f',
 VARIABLE -> 'g',
 VARIABLE -> 'h',
 VARIABLE -> 'i',
 VARIABLE -> 'j',
 VARIABLE -> 'k',
 VARIABLE -> 'l',
 VARIABLE -> 'm',
 VARIABLE -> 'n',
 VARIABLE -> 'o',
 VARIABLE -> 'p',
 VARIABLE -> 'q',
 VARIABLE -> 'r',
 VARIABLE -> 's',
 VARIABLE -> 't',
 VARIABLE -> 'u',
 VARIABLE -> 'v',
 VARIABLE -> 'w',
 VARIABLE -> 'x',
 VARIABLE -> 'y',
 VARIABLE -> 'z',
 DIGIT -> '0',
 DIGIT -> '1',
 DIGIT -> '2',
 DIGIT -> '3',
 DIGIT -> '4',
 DIGIT -> '5',
 DIGIT -> '6',
 DIGIT -> '7',
 DIGIT -> '8',
 DIGIT -> '9',
 DIGIT -> '10',
 DIGIT -> '11',
 DIGIT -> '12',
 DIGIT -> '13',
 DIGIT -> '14',
 DIGIT -> '15',
 DIGIT -> '16',
 DIGIT -> '17',
 DIGIT -> '18',
 DIGIT -> '19',
 DIGIT -> '20',
 ARITHMETIC_OPERATOR -> '+',
 ARITHMETIC_OPERATOR -> '-',
 ARITHMETIC_OPERATOR -> '*',
 ARITHMETIC_OPERATOR -> '/',
 RELATIONAL_OPERATOR -> '<',
 RELATIONAL_OPERATOR -> '>',
 RELATIONAL_OPERATOR -> '<=',
 R